In [5]:
import torch
import re
def xprint(s):
    s = str(s)
    s = re.sub(' *#.*','',s)
    print(s)

In [6]:
def myfn(x):
    y = x[0]
    for i in range(1, x.size(0)):
        y = y + x[i]
    return y

In [7]:
inp = torch.randn(5,5)
traced_fn = torch.jit.trace(myfn, inp)
print(traced_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  y0 = torch.add(y, torch.select(x, 0, 1), alpha=1)
  y1 = torch.add(y0, torch.select(x, 0, 2), alpha=1)
  y2 = torch.add(y1, torch.select(x, 0, 3), alpha=1)
  _0 = torch.add(y2, torch.select(x, 0, 4), alpha=1)
  return _0



In [8]:
scripted_fn = torch.jit.script(myfn)
print(scripted_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  _0 = torch.__range_length(1, torch.size(x, 0), 1)
  y0 = y
  for _1 in range(_0):
    i = torch.__derive_index(_1, 1, 1)
    y1 = torch.add(y0, torch.select(x, 0, i), alpha=1)
    y0 = y1
  return y0



In [9]:
xprint(scripted_fn.graph)

graph(%x.1 : Tensor):
  %10 : bool = prim::Constant[value=1]()
  %2 : int = prim::Constant[value=0]()
  %5 : int = prim::Constant[value=1]()
  %y.1 : Tensor = aten::select(%x.1, %2, %2)
  %7 : int = aten::size(%x.1, %2)
  %9 : int = aten::__range_length(%5, %7, %5)
  %y : Tensor = prim::Loop(%9, %10, %y.1)
    block0(%11 : int, %y.6 : Tensor):
      %i.1 : int = aten::__derive_index(%11, %5, %5)
      %18 : Tensor = aten::select(%x.1, %2, %i.1)
      %y.3 : Tensor = aten::add(%y.6, %18, %5)
      -> (%10, %y.3)
  return (%y)



In [10]:
from unet_wrapper import UNetWrapper

In [11]:
seg_dict = torch.load('model.state', map_location='cpu')
seg_model = UNetWrapper(in_channels=8, n_classes=1, depth=4, wf=3, padding=True,         
batch_norm=True, up_mode='upconv') 
seg_model.load_state_dict(seg_dict['model_state'])
seg_model.eval()
for p in seg_model.parameters():
    p.requires_grad_(False)

traced_seg_model = torch.jit.trace(seg_model, torch.randn((1, 8, 512, 512), requires_grad=True))
torch.jit.save(traced_seg_model, './src/batched_app/traced_seg_model.pt')

In [12]:
traced_seg_model

UNetWrapper(
  original_name=UNetWrapper
  (input_batchnorm): BatchNorm2d(original_name=BatchNorm2d)
  (unet): UNet(
    original_name=UNet
    (down_path): ModuleList(
      original_name=ModuleList
      (0): UNetConvBlock(
        original_name=UNetConvBlock
        (block): Sequential(
          original_name=Sequential
          (0): Conv2d(original_name=Conv2d)
          (1): ReLU(original_name=ReLU)
          (2): BatchNorm2d(original_name=BatchNorm2d)
          (3): Conv2d(original_name=Conv2d)
          (4): ReLU(original_name=ReLU)
          (5): BatchNorm2d(original_name=BatchNorm2d)
        )
      )
      (1): UNetConvBlock(
        original_name=UNetConvBlock
        (block): Sequential(
          original_name=Sequential
          (0): Conv2d(original_name=Conv2d)
          (1): ReLU(original_name=ReLU)
          (2): BatchNorm2d(original_name=BatchNorm2d)
          (3): Conv2d(original_name=Conv2d)
          (4): ReLU(original_name=ReLU)
          (5): BatchNorm2d(origi